In [1]:
import json
from tqdm import tqdm
from utils import (
    calc_iou, any_match,
    gen_froc_plot, gen_tabular_data,
    convert_to_tensor
)
from monai.data.box_utils import non_max_suppression
import torch
import pandas as pd

In [2]:
files = [
    'pixel-parsed-mixed_test-overjoyed-hen-305',
    'pixel-parsed-mixed_test-indecisive-eel-172',
    'pixel-parsed-mixed_test-powerful-goat-766',
    'pixel-parsed-mixed_test-crawling-sloth-537',
    'pixel-parsed-all'
]

In [3]:
all_fps_per_image = []
all_total_sensitivity = []
tabular_data = []
all_testy = []
all_lr_probs = []

composed_tp, composed_fp, composed_fn = 0, 0, 0
composed_tp_probs = []
composed_testy = []
composed_lr_probs = []
composed_fp_probs = []

wrong_ones = []

for file_name in files:
    output_path = '/data/output/validation/each_window/hc/'

    data_list_file_path = f'{output_path}{file_name}.json'

    f = open(data_list_file_path)
    data = json.load(f)

    iou_thresh = 0.5
    num_images = 21
    num_targets = 27

    fp_probs = []
    tp_probs = []
    tp, fp, fn = 0, 0, 0

    testy = []
    lr_probs = []

    for content in tqdm(data['test']):
        converted = convert_to_tensor(content['pred_box'])

        keep = non_max_suppression(
            converted, torch.tensor(content['score']), nms_thresh=0.1, max_proposals=100
        )

        boxes_after_nms = [content['pred_box'][index] for index in keep]
        scores_after_nms = torch.tensor(content['score'])[keep]

        for pred_box, score in zip(boxes_after_nms, scores_after_nms):
            # if score > 0.5:
            iou = 0
            if any_match(content['true_box'][0], pred_box):
                iou = calc_iou(content['true_box'][0], pred_box)
            if iou >= iou_thresh:
                tp += 1
                tp_probs.append(score)
                composed_tp += 1
                composed_tp_probs.append(score)

                testy.append(1)
                lr_probs.append(score)
                composed_testy.append(1)
                composed_lr_probs.append(score)
            if iou < iou_thresh and iou > 0:
                fp += 1
                fp_probs.append(score)
                composed_fp += 1
                composed_fp_probs.append(score)

                testy.append(0)
                lr_probs.append(score)
                composed_testy.append(0)
                composed_lr_probs.append(score)

                wrong_ones.append(content['image'])
            
            if iou == 0:
                fn += 1
                composed_fn += 1

                testy.append(0)
                lr_probs.append(score)
                composed_testy.append(0)
                composed_lr_probs.append(score)

                wrong_ones.append(content['image'])

    fps_per_image, total_sensitivity, precision, recall, p1, p2, p3, p4, p5, p6 ,p7, cpm = gen_tabular_data(
        tp, fp, fn, fp_probs, tp_probs, num_images, num_targets
    )

    all_fps_per_image.append(fps_per_image)
    all_total_sensitivity.append(total_sensitivity)
    all_testy.append(testy)
    all_lr_probs.append(lr_probs)

    tabular_data.append({
        "1/8": p1,
        "2/8": p2,
        "4/8": p3,
        "1": p4,
        "2": p5,
        "4": p6,
        "8": p7,
        "CPM": cpm
    })

first_table = pd.DataFrame(tabular_data)

100%|██████████| 21/21 [00:00<00:00, 39.72it/s]


to select the most "wrong" exam, all fp where printed, and the most frequent was selected

In [4]:
print(len(wrong_ones))

10322


In [5]:
wrong_ones[0:10]

['/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz',
 '/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz']

In [7]:
from collections import Counter

print(Counter(wrong_ones).keys())
print(Counter(wrong_ones).values()) # counts the elements' frequency

dict_keys(['/data/HC_Images_resample/PL155113087338785/PL155113087338785.nii.gz', '/data/HC_Images_resample/PL969252002286674/PL969252002286674.nii.gz', '/data/HC_Images_resample/PL153971807430236/PL153971807430236.nii.gz', '/data/HC_Images_resample/PL206802585657179/PL206802585657179.nii.gz', '/data/HC_Images_resample/PL316898027619487/PL316898027619487.nii.gz', '/data/HC_Images_resample/PL559798186095424/PL559798186095424.nii.gz', '/data/HC_Images_resample/PL421132963270314/PL421132963270314.nii.gz', '/data/HC_Images_resample/PL657447059971795/PL657447059971795.nii.gz', '/data/HC_Images_resample/PL245366064520387/PL245366064520387.nii.gz', '/data/HC_Images_resample/PL217160176826976/PL217160176826976.nii.gz', '/data/HC_Images_resample/PL620133077537220/PL620133077537220.nii.gz', '/data/HC_Images_resample/PL652297908013408/PL652297908013408.nii.gz', '/data/HC_Images_resample/PL926008951016659/PL926008951016659.nii.gz', '/data/HC_Images_resample/PL606729492858408/PL606729492858408.nii.